<a href="https://colab.research.google.com/github/liuyixin-louis/Teddy2020T/blob/master/2020%E6%B3%B0%E8%BF%AA%E6%9D%AF%E7%BD%91%E7%BB%9C%E6%90%AD%E5%BB%BA%E5%AE%9E%E9%AA%8C%E4%BB%A3%E7%A0%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 包\工具

In [0]:
!pip install jsonlines
import jsonlines

In [0]:
import re
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(".*?[。！？]")

In [0]:
!pip install 'kashgari>=1.0.0,<2.0.0'
%tensorflow_version 1.x
from kashgari.corpus import ChineseDailyNerCorpus
from kashgari.tasks.labeling import BiLSTM_Model
import kashgari
kashgari.config.use_cudnn_cell = True

     |████████████████████████████████| 92kB 7.4MB/s 
     |████████████████████████████████| 17.3MB 203kB/s 
  Created wheel for keras-gpt-2: filename=keras_gpt_2-0.14.0-cp36-none-any.whl size=10525 sha256=8bef5569b37589be2720e05e27f370944a247fe6295faeb5de76aad238e4a28e
  Stored in directory: /root/.cache/pip/wheels/ec/d8/06/ba8216a77a55b8ba4a5c3932c7df93e87eeaea83ced27822aa
  Created wheel for bert4keras: filename=bert4keras-0.6.5-cp36-none-any.whl size=34212 sha256=d2eb5a14813d93c6379dfd99435b187832ab5451f646fec5fb283da3f64ab48d
  Stored in directory: /root/.cache/pip/wheels/97/82/72/56a894ccb2337a25b679cbac552bf2f15f5e8e798a37313de6
  Created wheel for keras-bert: filename=keras_bert-0.81.0-cp36-none-any.whl size=37913 sha256=e444ffdaa3b5fbca6109153b72f8a3cb4eda96fdae50f42079f231ed00f4cce2
  Stored in directory: /root/.cache/pip/wheels/bd/27/da/ffc2d573aa48b87440ec4f98bc7c992e3a2d899edb2d22ef9e
  Created wheel for keras-transformer: filename=keras_transformer-0.33.0-cp36-none-any.w

TensorFlow 1.x selected.


In [0]:
def get_entry(res):
  ar = []
  for ri in res:
    for ai in ri['labels']:
      if ai['entity'] == 'address':
        ar.append(ai['value'])
  return ar

def makedata(s):
  res = []
  for i in s:
    res.append(i)
  return [res]

def visualizeResult(x,y):
  for xi,yi in zip(x,y):
    for i in range(len(xi)):
      if yi[i]!='O':
        print (xi[i]+yi[i])
import json
def get_data(examples):
  x = []
  y = []
  for e in examples:
    xi = e['context'].split(' ')
    x.append(xi)
    yi = e['tag'].split(' ')
    y.append(yi)
  for i in range(len(y)):
    for j in range(len(y[i])):
      if y[i][j] == 'B-address' or y[i][j] == 'I-address':
        pass
      else:
        y[i][j] = 'O'
  return x,y
def create_datas(input_path,mode):
    examples = []
    with open(input_path, 'r') as f:
        idx = 0
        for line in f:
            json_d = {}
            line = json.loads(line.strip())
            text = line['text']
            label_entities = line.get('label', None)
            words = list(text)
            labels = ['O'] * len(words)
            if label_entities is not None:
                for key, value in label_entities.items():
                    for sub_name, sub_index in value.items():
                        for start_index, end_index in sub_index:
                            assert ''.join(words[start_index:end_index + 1]) == sub_name
                            if start_index == end_index:
                                labels[start_index] = 'S-' + key
                            else:
                                labels[start_index] = 'B-' + key
                                labels[start_index + 1:end_index + 1] = ['I-' + key] * (len(sub_name) - 1)
            json_d['id'] = f"{mode}_{idx}"
            json_d['context'] = " ".join(words)
            json_d['tag'] = " ".join(labels)
            json_d['raw_context'] = "".join(words)
            idx += 1
            examples.append(json_d)
    x,y = get_data(examples)

    return x,y


# 数据读取

In [0]:
import pandas as pd
content = pd.read_csv('/content/drive/My Drive/正常URL数据.csv')

In [0]:
content

,Unnamed: 0,url,content,sname,images,methods,ocr_text,content_count,ocr_text_count,all_text,all_text_count
0,0,https://mp.weixin.qq.com/s/N76fZ6R4zeiB6Zqa3TYaxg,\n。据广东省卫健委今日（4日）最新消息：截至2月3日24时，。全省累计报告新型冠状病毒感染...,南方都市报,['https://mmbiz.qpic.cn/mmbiz_png/3OEpTPib0kVi...,template,南方都市报\nDD A\n\t广东新型冠状病毒感染肺炎\n确诊病例截2月3日24时A南都\n...,13999,869,。。据广东省卫健委今日（4日）最新消息：截至2月3日24时，。全省累计报告新型冠状病毒感染的...,14868
1,1,http://www.gz.gov.cn/zt/qlyfdyyqfkyz/qktb/cont...,2020年2月3日0时至24时，广州市报告新型冠状病毒感染的肺炎新增确诊病例27例，其中越秀...,截至2020年2月3日24时广州市新型冠状病毒感染的肺炎疫情情况,['http://www.gz.gov.cn/img/0/40/40840/5650490....,GNE,0\n≤34-1011-2021-30314041\n1-6061-707180281\n图...,1087,59,2020年2月3日0时至24时，广州市报告新型冠状病毒感染的肺炎新增确诊病例27例，其中越秀...,1146
2,2,http://wjw.gz.gov.cn/ztzl/xxfyyqfk/yqtb/conten...,2020年2月3日0时至24时，广州市报告新型冠状病毒感染的肺炎新增确诊病例27例，其中越秀...,广州市卫生健康委员会网站,['http://wjw.gz.gov.cn/img/0/40/40777/5650395....,GNE,0\n≤34-1011-2021-30314041\n1-6061-707180281\n图...,1091,59,2020年2月3日0时至24时，广州市报告新型冠状病毒感染的肺炎新增确诊病例27例，其中越秀...,1150
3,3,https://mp.weixin.qq.com/s/KkYpkD4oV997_ZPNRa0vHg,\n。据广东省卫健委官网，截至1月31日12时，全省累计报告新型冠状病毒感染的肺炎。确诊病例...,南方都市报,['https://mmbiz.qpic.cn/mmbiz_png/3OEpTPib0kVi...,template,南方都市报\nDD A\n\t广东新型冠状病毒感染肺炎\n确诊病例截1月31日12时AO南\...,3651,438,。。据广东省卫健委官网，截至1月31日12时，全省累计报告新型冠状病毒感染的肺炎。确诊病例4...,4089
4,4,http://wjw.sz.gov.cn/yqxx/202002/t20200203_189...,截至2月2日24时，深圳的“新型肺炎”病例累计达226例。\n2月2日，疾控专业人员通过最新...,最新！增27个小区！深圳确诊病人到过这些地方,['http://wjw.sz.gov.cn/W020200203518286875951....,GNE,NaN,900,0,截至2月2日24时，深圳的“新型肺炎”病例累计达226例。。2月2日，疾控专业人员通过最新的...,900
...,...,...,...,...,...,...,...,...,...,...,...
172,172,http://www.jiangmen.gov.cn/home/zwyw/content/p...,江门日报讯 （江讯） 1月29日，省卫健委公布了我市首例新型冠状病毒感染的肺炎确诊病例。昨日...,我市首例确诊病例接触者排查进展顺利 密切接触者有7人,[],GNE,NaN,2305,0,江门日报讯 （江讯） 1月29日，省卫健委公布了我市首例新型冠状病毒感染的肺炎确诊病例。昨日...,2305
173,173,http://wsjkj.zhuhai.gov.cn/zwgk/tzgg/content/p...,2月5日16：00，广东省卫生健康委公布，珠海市新增3例新型冠状病毒感染的肺炎确诊病例。截至...,珠海市卫生健康局网站,[],GNE,NaN,554,0,2月5日16：00，广东省卫生健康委公布，珠海市新增3例新型冠状病毒感染的肺炎确诊病例。截至...,554
174,174,http://wjw.hefei.gov.cn/ztzl/xxgzbdgrdfyyqfk/x...,为进一步加强救治和防控工作，现将确诊病例救治医院、性别年龄、涉及小区（或村居）等有关分布情况...,2月5日合肥市新型冠状病毒感染的肺炎疫情确诊病例有关分布情况通报,[],GNE,NaN,1221,0,为进一步加强救治和防控工作，现将确诊病例救治医院、性别年龄、涉及小区（或村居）等有关分布情况...,1221
175,175,http://wjw.hefei.gov.cn/ztzl/xxgzbdgrdfyyqfk/x...,为进一步加强救治和防控工作，现将确诊病例救治医院、性别年龄、涉及小区（或村居）等有关分布情况...,2月6日合肥市新型冠状病毒感染的肺炎疫情确诊病例有关分布情况通报,[],GNE,NaN,1287,0,为进一步加强救治和防控工作，现将确诊病例救治医院、性别年龄、涉及小区（或村居）等有关分布情况...,1287


In [0]:
def mkdir(path):
    # 引入模块
    import os
 
    # 去除首位空格
    path=path.strip()
    # 去除尾部 \ 符号
    path=path.rstrip("\\")
 
    # 判断路径是否存在
    # 存在     True
    # 不存在   False
    isExists=os.path.exists(path)
 
    # 判断结果
    if not isExists:
        # 如果不存在则创建目录
        # 创建目录操作函数
      os.makedirs(path) 
 
        # print path+' 创建成功'
      return True
    else:
      # 如果目录存在则不创建，并提示目录已存在
      # print path+' 目录已存在'
      return False
 
# # 定义要创建的目录
# mkpath="d:\\qttc\\web\\"
# # 调用函数
# mkdir(mkpath)

In [0]:
def reformate(s,filename):
    f = open(filename,'w+')
    f.write('')
    f.close
    tokenizer = RegexpTokenizer(".*?[。！？]")
    fenjuzi = tokenizer.tokenize(s)
    
    with jsonlines.open(filename, mode='w') as writer:
#         writer.write({'a':5})
        for i in range(len(fenjuzi)):
            writer.write({'id':i,'text':fenjuzi[i]})


In [0]:

for i in range(len(content)):
  reformate(content['all_text'][i],'/content/drive/My Drive/泰迪杯/测试/'+str(i)+'.json')

In [0]:
s = ''
with open('/content/drive/My Drive/Allcontent57.txt','r+',encoding='utf-8')as f:
  s = f.read()

import jieba
import re
si = re.split('。|！|\!|\.|？|\?',s)

for i in range(len(si)):
  try:
    si[i] = si[i].replace('\n','#')
  except Exception as e:
    print(e)
for i in range(len(si)):
  si[i] = list(si[i])

our_ploblem = si[:]
len(si)

In [0]:
train_x,train_y = create_datas('/content/drive/My Drive/ourdata/NERclue2020/train.json','train')
dev_x,dev_y = create_datas('/content/drive/My Drive/ourdata/NERclue2020/dev.json','dev')

In [0]:
len(train_x)

2095

In [0]:
len(dev_x)

273

In [0]:
path  = '/content/drive/My Drive/ourdata/allin_test.test'
ws = []
ls = []
s= ''
try:
  f = open(path,'r+',encoding="gbk")
  s = f.read()
except Exception as e:
  f = open(path,'r+',encoding="utf-8")
  s = f.read()
f.close()
d = s.split('\n')
wt = []
lt= []
for di in d:
  sbi = di.split(' ')
  if len(sbi) == 2:
    w,l = sbi
    wt.append(w)
    lt.append(l)
    if w == '。' or w == '？':
      if 'B-address' in lt or 'I-address' in lt:
        ws.append(wt)
        ls.append(lt)
      wt = []
      lt= []
      
test_x_our = ws
test_y_our = ls

# 实验一：各种模型在cluener数据集上的表现

## 无预训练词嵌入+bilstm(- acc: 0.9813 - val_loss: 0.3156 - val_acc: 0.9341 过拟合)

In [0]:
import kashgari
from kashgari.tasks.labeling import BiLSTM_Model
model1 = BiLSTM_Model()

In [0]:
model1.fit(train_x, train_y, dev_x, dev_y, epochs=50)
model1.save('/content/drive/My Drive/shiyan/01/baseEmbacring+bilstm')

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 50)]              0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 50, 100)           197900    
_________________________________________________________________
layer_blstm (Bidirectional)  (None, 50, 256)           235520    
_________________________________________________________________
layer_dropout (Dropout)      (None, 50, 256)           0         
_________________________________________________________________
layer_time_distributed (Time (None, 50, 4)             1028      
_________________________________________________________________
activation (Activation)      (None, 50, 4)             0         
Total params: 434,448
Trainable params: 434,448
Non-trainable params: 0
_____________________________________________________

In [0]:
model = kashgari.utils.load_model('/content/drive/My Drive/shiyan/01/baseEmbacring+bilstm')

In [0]:
model.evaluate(test_x_our,test_y_our)

           precision    recall  f1-score   support

  address     0.0000    0.0000    0.0000        94

micro avg     0.0000    0.0000    0.0000        94
macro avg     0.0000    0.0000    0.0000        94



'           precision    recall  f1-score   support\n\n  address     0.0000    0.0000    0.0000        94\n\nmicro avg     0.0000    0.0000    0.0000        94\nmacro avg     0.0000    0.0000    0.0000        94\n'

In [0]:
test_x,test_y = create_datas('/content/drive/My Drive/ourdata/weijianwei.json','test')
res = model.predict_entities(test_x)
get_entry(res)

['太 原 南',
 '临 汾',
 '临 汾 西',
 '太 原 市',
 '太 原 南',
 '阳 泉 市',
 '盂 县',
 '秀 水 镇',
 '路 洞',
 '区',
 '香 乐 村',
 '万 福 隆 超 市',
 '山 西 省',
 '平 遥 县',
 '香 乐 乡',
 '香 乐 村',
 '尖 草 坪 区',
 '汇 丰 街 道',
 '兴 华 北 小 区',
 '武 汉',
 '平 遥',
 '武 汉',
 '武 汉',
 '太 原',
 '太 原',
 '太 原',
 '临 汾 市',
 '吉 县',
 '苑',
 '尖 草 坪 区',
 '汇 丰 街 道',
 '兴 华 北 小 区',
 '沁 水 县',
 '郑 庄 镇',
 '中 乡 村',
 '临',
 '吉 高 速',
 '吉 县',
 '平 遥 县']

## baseEmbacring+bilstm_crf（50epoches,best f1:0.0800 效果一般）

In [0]:
import kashgari
from kashgari.tasks.labeling import BiLSTM_CRF_Model
model = BiLSTM_CRF_Model()

In [0]:
model.fit(train_x, train_y, dev_x, dev_y, epochs=50)

In [0]:
model = kashgari.utils.load_model('/content/drive/My Drive/shiyan/01/baseEmbacring+bilstm_crf')

/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:22: UserWarning: bert4keras.bert has been renamed as bert4keras.models.
  warnings.warn('bert4keras.bert has been renamed as bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:23: UserWarning: please use bert4keras.models.
  warnings.warn('please use bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:44: UserWarning: bert4keras.tokenizer has been renamed as bert4keras.tokenizers.
  warnings.warn('bert4keras.tokenizer has been renamed as bert4keras.tokenizers.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:45: UserWarning: please use bert4keras.tokenizers.
  warnings.warn('please use bert4keras.tokenizers.')


In [0]:
model.evaluate(test_x_our,test_y_our)

           precision    recall  f1-score   support

  address     0.0642    0.1061    0.0800        66

micro avg     0.0642    0.1061    0.0800        66
macro avg     0.0642    0.1061    0.0800        66



'           precision    recall  f1-score   support\n\n  address     0.0642    0.1061    0.0800        66\n\nmicro avg     0.0642    0.1061    0.0800        66\nmacro avg     0.0642    0.1061    0.0800        66\n'

In [0]:
test_x,test_y = create_datas('/content/drive/My Drive/ourdata/weijianwei.json','test')
res = model.predict_entities(test_x)
get_entry(res)

['临 汾 西',
 '号',
 '西',
 '太 原 市',
 '至 太 原 南 站',
 '观',
 '参 加 本 村',
 '本 村',
 '吉 县',
 '阳 泉 市 盂 县 秀 水 镇 路 洞 社 区',
 '红 楼 干',
 '香 乐 村 万 福 隆 超 市',
 '山 西 省',
 '遥 县 香 乐 乡 香 乐 村',
 '尖 草 坪 区 汇 丰 街 道 兴 华 北 小 区',
 '武 汉',
 '沁 水 县',
 '爷 爷',
 '吉 县 人 民 医 院 发 热 门',
 '太 原 市 第 四 人 民 医 院 住 院',
 '武 汉 返',
 '武 汉',
 '太 原',
 '班',
 '盂 县 人 民 医 院 隔 离 医 学 观',
 '山 西 万 峰 医',
 '市 吉 县 荣 辰 佳 苑',
 '尖 草 坪 区 汇 丰 街 道 兴 华 北 小 区',
 '汾 市',
 '第 三 人 民 医 院 （ 市 级 定 点 医',
 '沁 水 县 郑 庄 镇 中 乡 村',
 '吉 高 速 回',
 '吉 县',
 '县',
 '医 院 1 2 0 救 护',
 '平 遥 县 香 乐 乡']

### 忽略：无预训练词嵌入+bilstm+crf（50epoches,best f1:0.6209 效果一般）

In [0]:
!wget https://storage.googleapis.com/cluebenchmark/tasks/cluener_public.zip 

--2020-05-07 10:42:11--  https://storage.googleapis.com/cluebenchmark/tasks/cluener_public.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.76.128, 2a00:1450:400c:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.76.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1054686 (1.0M) [application/zip]
Saving to: ‘cluener_public.zip.1’

cluener_public.zip. 100%[===================>]   1.00M  --.-KB/s    in 0.01s   

2020-05-07 10:42:11 (71.0 MB/s) - ‘cluener_public.zip.1’ saved [1054686/1054686]



In [0]:
! unzip /content/cluener_public.zip -d ./cluener_public/

Archive:  /content/cluener_public.zip
  inflating: ./cluener_public/train.json  
  inflating: ./cluener_public/dev.json  
  inflating: ./cluener_public/test.json  
  inflating: ./cluener_public/cluener_predict.json  
  inflating: ./cluener_public/README.md  


In [0]:
! git clone  https://github.com/CLUEbenchmark/CLUENER2020/

Cloning into 'CLUENER2020'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 352 (delta 0), reused 15 (delta 0), pack-reused 333
Receiving objects: 100% (352/352), 890.81 KiB | 2.24 MiB/s, done.
Resolving deltas: 100% (162/162), done.


In [0]:
!pip install jsonlines

In [0]:
import os
path = "/content/CLUENER2020/bilstm_crf_pytorch"
os.chdir(path)

In [0]:
!python download_clue_data.py --data_dir=./dataset --tasks=cluener

	Completed! Downloaded cluener data to directory ./dataset/cluener


In [0]:
!python run_lstm_crf.py --do_train

In [0]:
!python run_lstm_crf.py --do_eval

In [0]:
!python run_lstm_crf.py --do_predict

In [0]:
visualizeResult(x_data,y)

广B-LOC
东I-LOC
省I-LOC
深B-LOC
圳I-LOC
市I-LOC
宝B-LOC
安I-LOC
区I-LOC
西I-LOC
乡I-LOC


## bert+BiLSTM_CRF_Model（50epoches,best f1:0.6693 效果较好）

In [0]:
!wget https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip
!unzip chinese_L-12_H-768_A-12.zip -d chinese_L-12_H-768_A-12


--2020-05-09 20:47:30--  https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.194.128, 2404:6800:4003:c04::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.194.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 381892918 (364M) [application/zip]
Saving to: ‘chinese_L-12_H-768_A-12.zip’

chinese_L-12_H-768_ 100%[===================>] 364.20M  49.0MB/s    in 8.2s    

2020-05-09 20:47:39 (44.5 MB/s) - ‘chinese_L-12_H-768_A-12.zip’ saved [381892918/381892918]

Archive:  chinese_L-12_H-768_A-12.zip
   creating: chinese_L-12_H-768_A-12/chinese_L-12_H-768_A-12/
  inflating: chinese_L-12_H-768_A-12/chinese_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: chinese_L-12_H-768_A-12/chinese_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: chinese_L-12_H-768_A-12/chinese_L-12_H-768_A-12/vocab.txt  
  inflating: chinese_L-12_H-768_A

In [0]:
import kashgari
from kashgari.tasks.labeling import BiLSTM_CRF_Model
from kashgari.embeddings import BERTEmbedding

bert_embed = BERTEmbedding('/content/chinese_L-12_H-768_A-12/chinese_L-12_H-768_A-12',
                           task=kashgari.LABELING,
                           sequence_length=100)
model = BiLSTM_CRF_Model(bert_embed)

/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:22: UserWarning: bert4keras.bert has been renamed as bert4keras.models.
  warnings.warn('bert4keras.bert has been renamed as bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:23: UserWarning: please use bert4keras.models.
  warnings.warn('please use bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:44: UserWarning: bert4keras.tokenizer has been renamed as bert4keras.tokenizers.
  warnings.warn('bert4keras.tokenizer has been renamed as bert4keras.tokenizers.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:45: UserWarning: please use bert4keras.tokenizers.
  warnings.warn('please use bert4keras.tokenizers.')


In [0]:
model.fit(train_x,
          train_y,
          x_validate=dev_x,
          y_validate=dev_y,
          epochs=50,
          batch_size=512)
# model.save('/content/drive/My Drive/shiyan/01/bert_bilstm_crf.h5')

/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:22: UserWarning: bert4keras.bert has been renamed as bert4keras.models.
  warnings.warn('bert4keras.bert has been renamed as bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:23: UserWarning: please use bert4keras.models.
  warnings.warn('please use bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:44: UserWarning: bert4keras.tokenizer has been renamed as bert4keras.tokenizers.
  warnings.warn('bert4keras.tokenizer has been renamed as bert4keras.tokenizers.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:45: UserWarning: please use bert4keras.tokenizers.
  warnings.warn('please use bert4keras.tokenizers.')


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 100)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 100, 768), ( 16226304    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 100, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

KeyboardInterrupt: ignored

In [0]:
bert = kashgari.utils.load_model('/content/drive/My Drive/shiyan/01/+bert')

/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:22: UserWarning: bert4keras.bert has been renamed as bert4keras.models.
  warnings.warn('bert4keras.bert has been renamed as bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:23: UserWarning: please use bert4keras.models.
  warnings.warn('please use bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:44: UserWarning: bert4keras.tokenizer has been renamed as bert4keras.tokenizers.
  warnings.warn('bert4keras.tokenizer has been renamed as bert4keras.tokenizers.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:45: UserWarning: please use bert4keras.tokenizers.
  warnings.warn('please use bert4keras.tokenizers.')


In [0]:
model.evaluate(dev_x, dev_y)

           precision    recall  f1-score   support

  address     0.6632    0.6756    0.6693       373

micro avg     0.6632    0.6756    0.6693       373
macro avg     0.6632    0.6756    0.6693       373



'           precision    recall  f1-score   support\n\n  address     0.6632    0.6756    0.6693       373\n\nmicro avg     0.6632    0.6756    0.6693       373\nmacro avg     0.6632    0.6756    0.6693       373\n'

In [0]:
test_x,test_y = create_datas('/content/drive/My Drive/ourdata/weijianwei.json','test')

In [0]:
test_x

[['1',
  '5',
  '时',
  '4',
  '2',
  '分',
  '乘',
  '坐',
  '太',
  '原',
  '南',
  '-',
  '临',
  '汾',
  '西',
  'D',
  '9',
  '2',
  '0',
  '5',
  '车',
  '次',
  '（',
  '座',
  '位',
  '号',
  '0',
  '2',
  '.',
  '1',
  '6',
  'C',
  '）',
  '，',
  '1',
  '7',
  ':',
  '0',
  '7',
  '到',
  '达',
  '临',
  '汾',
  '西',
  '站',
  '。'],
 ['2',
  '0',
  '2',
  '0',
  '年',
  '1',
  '月',
  '2',
  '2',
  '日',
  '下',
  '午',
  '，',
  '在',
  '太',
  '原',
  '市',
  '乘',
  '坐',
  '8',
  '4',
  '9',
  '路',
  '公',
  '交',
  '车',
  '至',
  '太',
  '原',
  '南',
  '站',
  '。'],
 ['2',
  '0',
  '2',
  '0',
  '年',
  '2',
  '月',
  '5',
  '日',
  '居',
  '家',
  '隔',
  '离',
  '医',
  '学',
  '观',
  '察',
  '。'],
 ['2',
  '0',
  '2',
  '0',
  '年',
  '1',
  '月',
  '2',
  '2',
  '日',
  '1',
  '2',
  '时',
  '和',
  '2',
  '3',
  '日',
  '1',
  '9',
  '时',
  '两',
  '次',
  '参',
  '加',
  '本',
  '村',
  '居',
  '民',
  '家',
  '宴',
  '。'],
 ['2',
  '0',
  '2',
  '0',
  '年',
  '1',
  '月',
  '2',
  '4',
  '日',
  '-',
  '2',
  '9',
  '日',
  '去',
  

In [0]:
res = bert.predict_entities(test_x)

In [0]:
res[1]['labels']

[{'end': 16, 'entity': 'address', 'start': 14, 'value': '太 原 市'},
 {'end': 30, 'entity': 'address', 'start': 27, 'value': '太 原 南 站'}]

In [0]:
get_entry(res)

['太 原 南',
 '临 汾 西',
 '临 汾 西 站',
 '太 原 市',
 '太 原 南 站',
 '村',
 '吉 县 人 民 医 院',
 '阳 泉 市 盂 县 秀 水 镇 路 洞 社 区 （ 红 楼 干 休 所',
 '香 乐 村 万 福 隆 超 市',
 '山 西 省',
 '平 遥 县 香 乐 乡 香 乐 村',
 '尖 草 坪 区 汇 丰 街 道 兴 华 北 小 区',
 '武 汉',
 '平 遥',
 '沁 水 县',
 '吉 县 人 民 医 院',
 '太 原 市 第 四 人 民 医 院',
 '武 汉',
 '武 汉',
 '太 原',
 '太 原',
 '太 原',
 '盂 县 人 民 医 院',
 '山 西 万 峰 医 院',
 '临 汾 市 吉 县 荣 辰 佳 苑',
 '尖 草 坪 区 汇 丰 街 道 兴 华 北 小 区',
 '临 汾 市',
 '沁 水 县 郑 庄 镇 中 乡 村',
 '临 吉 高 速',
 '吉 县',
 '平 遥 县 人 民 医 院',
 '县 人 民 医 院 发 热 门 诊',
 '平 遥 县 香 乐 乡 派 出 所']

In [0]:
test_x,test_y = create_datas('/content/drive/My Drive/ourdata/shortest.json','test')
res = model.predict_entities(test_x)
get_entry(res)

['上 海',
 '尚 海 湾 豪 庭',
 '宝 山 区',
 '。',
 '徐 汇 区',
 '市 卫 健 委',
 '四 季 绿 城 （ 北 区 ）',
 '健 委',
 '虹 桥 花 苑',
 '城 市 超 市',
 '虹 梅 路 店',
 '虹 六 菜 市 场',
 '闵 行 区']

In [0]:
test_x,test_y = create_datas('/content/drive/My Drive/ourdata/weijianwei.json','test')
res = bert.predict_entities(test_x)
get_entry(res)

['太 原 南',
 '临 汾 西',
 '临 汾 西 站',
 '太 原 市',
 '太 原 南 站',
 '村',
 '吉 县 人 民 医 院',
 '阳 泉 市 盂 县 秀 水 镇 路 洞 社 区 （ 红 楼 干 休 所',
 '香 乐 村 万 福 隆 超 市',
 '山 西 省',
 '平 遥 县 香 乐 乡 香 乐 村',
 '尖 草 坪 区 汇 丰 街 道 兴 华 北 小 区',
 '武 汉',
 '平 遥',
 '沁 水 县',
 '吉 县 人 民 医 院',
 '太 原 市 第 四 人 民 医 院',
 '武 汉',
 '武 汉',
 '太 原',
 '太 原',
 '太 原',
 '盂 县 人 民 医 院',
 '山 西 万 峰 医 院',
 '临 汾 市 吉 县 荣 辰 佳 苑',
 '尖 草 坪 区 汇 丰 街 道 兴 华 北 小 区',
 '临 汾 市',
 '沁 水 县 郑 庄 镇 中 乡 村',
 '临 吉 高 速',
 '吉 县',
 '平 遥 县 人 民 医 院',
 '县 人 民 医 院 发 热 门 诊',
 '平 遥 县 香 乐 乡 派 出 所']

In [0]:
res[0]['labels'][0]['entity']

'address'

In [0]:
daan = []
for i in range(len(content)):
  xi = create_datas('/content/drive/My Drive/泰迪杯/测试/'+str(i)+'.json','test')
  res1 = bert.predict_entities(xi)
  reslist = get_entry(res1)
  res2 = '\n'.join(reslist)
  daan.append(res2)
content['Answer'] = daan

In [0]:
content['Answer']

0      广 东 省 卫 健 委\n佛 山 市\n阳 江 市\n清 远 市\n梅 州 市\n江 门 市...
1      广 州 市\n越 秀 区\n海 珠 区\n白 云 区\n天 河 区\n番 禺 区\n增 城 ...
2      广 州 市\n越 秀 区\n海 珠 区\n白 云 区\n天 河 区\n番 禺 区\n增 城 ...
3      广 东 省\n省\n广 州 市\n深 圳 市\n珠 海 市\n佛 山 市\n惠 州 市\n江...
4      深 圳\n深 圳\n深 圳 市 第 三 人 民 医 院\n社 区 基 层\n病 人 住 所\...
                             ...                        
172    江 门\n江 门 市\n湖 北 省 孝 感 市\n江 门 市 蓬 江 区 海 逸 城 邦\n...
173    珠 海 市\n湖 北 省 武 汉 市\n武 汉\n珠 海\n斗 门 区\n遵 医 五 院\n...
174    徽 省 立 医 院\n市 滨 湖 医 院\n安 医 大 附 属 巢 湖 医 院\n肥 东 县...
175    徽 省 立 医 院\n市 滨 湖 医 院\n安 医 大 附 属 巢 湖 医 院\n肥 东 县...
176    吉 林 市\n吉 林 市\n吉 林 市 中 心 医 院\n吉 林 市 传 染 病 医 院\n...
Name: Answer, Length: 177, dtype: object

In [0]:
content.to_csv('./result.csv')

In [0]:
simC = content[['url','Answer','sname']]

In [0]:
simC.to_csv('./simple_result.csv')

In [0]:
import json 
# Reading data back
with open('/content/drive/My Drive/泰迪杯/area.json', 'r+',encoding='utf-8-sig') as f:
  data = json.loads(f.read())  

In [0]:
'观澜街道库坑中心区' in content['content'][0]

True

In [0]:
def texsmartAPI(x):
  # -*- coding: utf8 -*-
  import json
  import requests

  obj = {"str": str(x)}
  req_str = json.dumps(obj).encode()

  url = "https://texsmart.qq.com/api"
  r = requests.post(url, data=req_str)
  r.encoding = "utf-8"
  return r.json()
  # print(r.text)
  #print(json.loads(r.text))

In [0]:
texsmartAPI('广州')

{'entity_list': [{'flag': 1,
   'hit': [0, 2, 0, 2],
   'meaning': {'related': ['上海',
     '江苏',
     '广东',
     '浙江',
     '北京',
     '山东',
     '天津',
     '四川',
     '安徽',
     '重庆']},
   'str': '广州',
   'tag': 'loc.city',
   'tag_i18n': '城市'}],
 'header': {'core_time_cost_ms': 0.616,
  'ret_code': 'succ',
  'time_cost': 0.000664,
  'time_cost_ms': 0.664},
 'norm_str': '广州',
 'phrase_list': [{'hit': [0, 2, 0, 2], 'str': '广州', 'tag': 'NR'}],
 'srl_str': '',
 'syntactic_parsing_str': '',
 'word_list': [{'hit': [0, 2, 0, 2], 'str': '广州', 'tag': 'NR'}]}

In [0]:
def handle_Text(x):
  l = x.split('\n')
  res = []
  for li in l:
    if len(li)<=2:
      continue
    apires = texsmartAPI(li)['entity_list']
    # elif apires['entity_list'][0]['tag'] == 'loc.city' and (len(apires['entity_list'][0]['str'])/len("广州") > 0.9):
    #   continue
    flag = False
    for apiresi in apires:
      if apiresi['tag_i18n'] == '地点':
        flag = True
        break
    if flag is True:
     res.appen(li)
  
  return "\n".join(res)

In [0]:
content['test'] = content['all_text'].apply(lambda x:handle_Text(x))

In [0]:
content['test']

0       
1       
2       
3       
4       
      ..
172     
173     
174     
175     
176     
Name: test, Length: 177, dtype: object

In [0]:
# test_x,test_y = create_datas('/content/drive/My Drive/ourdata/weijianwei.json','test')
res = bert.predict_entities(our_ploblem)
locationlist = get_entry(res)

In [0]:
locationlist

In [0]:
import pickle
# c=[1,2,3,4,5,6,7]
# a=c*4
f=open('./location_all.plk','wb')
pickle.dump(locationlist,f)
f.close()
# print(a)

In [0]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/content_withOCR_05061540.csv')

In [0]:
ress = []
for di in range(len(df)):
  text = df['all_text'][di]
  # s = ''
  # with open('/content/drive/My Drive/Allcontent57.txt','r+',encoding='utf-8')as f:
  #   s = f.read()

  import jieba
  import re
  texti = re.split('。|！|\!|\.|？|\?',text)

  for i in range(len(texti)):
    try:
      texti[i] = texti[i].replace('\n','#')
    except Exception as e:
      print(e)
  for i in range(len(texti)):
    texti[i] = list(texti[i])
  res = bert.predict_entities(texti)
  textilocationlist = get_entry(res)
  ress.append(textilocationlist)

In [0]:
ress

[['#',
  '最',
  '新',
  '消',
  '息',
  '肇 庆 市',
  '肇 庆 市',
  '封 开 县',
  '端 州 区',
  '鼎 湖 区',
  '高 要 区',
  '四 会 市',
  '广 宁 县',
  '封 开 县',
  '其',
  '湖 北',
  '武 汉',
  '\xa0',
  '肇 庆 市',
  '1',
  '1',
  '封 开',
  '封 开 县 江 口 镇 河 堤 二 路 2 2 号',
  '湖 北 省',
  '封 开 县',
  '封 开 县',
  '好 又 多',
  '封 开 县',
  '封 开 县 江 口 镇 河 堤 二 路 2 2 号 、 江 口 镇 蝴 蝶 塘 路 封 开 县 曲 艺 社',
  '好 又 多 超 市',
  '水 上 居 民 安 置 楼',
  '封 开',
  '封 开 县 江 口 镇 水 上 居 民 安 置 楼',
  '武 汉',
  '江 口 镇 星 辉 曲 艺 社 江 口 社 区',
  '宾 悦 酒 店 2 楼',
  '平 凤 镇 新 围 村',
  '封 开 县 江 口 镇 杏 花 宾 馆',
  '肇 庆 市',
  '\xa0',
  '封 开',
  '封 开 县 江 口 镇 东 方 二 路 1 1 5 号',
  '江 口 镇',
  '苏 某',
  '江 口 镇 宾 悦 酒 店 二 楼',
  '封 开 县 江 口 镇 东 方 二 路 1 1 5 号',
  '井 根 农 贸 市 场',
  '江 口 镇 东 堤 路 2 5 号 亲 戚 家',
  '杏 花 宾 馆 宿 舍',
  '苏 某',
  '店 铺',
  '武 汉',
  '肇 庆 市',
  '肇 庆 市',
  '\xa0',
  '封 开 县 江 口 镇',
  '乐 惠 士 多 店',
  '曙 光 路 3 1 号',
  '封 开 县 广 信 嘉 园',
  '封 开 县 唐 八 叔 清 补 羊 火 锅 店',
  '武 汉',
  '\xa0',
  '湖 北 省 黄 冈 市',
  '武 汉',
  '广 州',
  '广 州 滘 口 汽 车 站',
  '封 开 县 杏 花 镇 新 和 村',
  '庆 市',
  '\xa0',
  '广 州',

In [0]:
s = ''

In [0]:
model.save('/content/drive/My Drive/shiyan/01/+bert/')

In [0]:
bertmoedl = kashgari.utils.load_model('/content/drive/My Drive/shiyan/01/+bert')

In [0]:
bertmoedl.evaluate(test_x_our,test_y_our)

           precision    recall  f1-score   support

  address     0.2157    0.2340    0.2245        94

micro avg     0.2157    0.2340    0.2245        94
macro avg     0.2157    0.2340    0.2245        94



'           precision    recall  f1-score   support\n\n  address     0.2157    0.2340    0.2245        94\n\nmicro avg     0.2157    0.2340    0.2245        94\nmacro avg     0.2157    0.2340    0.2245        94\n'

## ernie1.0xxx

In [0]:
!wget https://ernie.bj.bcebos.com/ERNIE_1.0_max-len-512.tar.gz

--2020-05-07 10:51:39--  https://ernie.bj.bcebos.com/ERNIE_1.0_max-len-512.tar.gz
Resolving ernie.bj.bcebos.com (ernie.bj.bcebos.com)... 103.235.46.61
Connecting to ernie.bj.bcebos.com (ernie.bj.bcebos.com)|103.235.46.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 367271417 (350M) [application/x-gzip]
Saving to: ‘ERNIE_1.0_max-len-512.tar.gz’

ERNIE_1.0_max-len-5 100%[===================>] 350.26M  9.36MB/s    in 94s     

2020-05-07 10:53:16 (3.72 MB/s) - ‘ERNIE_1.0_max-len-512.tar.gz’ saved [367271417/367271417]



In [0]:
!tar zxvf ERNIE_1.0_max-len-512.tar.gz 

ernie_config.json
vocab.txt
params/
params/encoder_layer_11_ffn_fc_0.w_0
params/encoder_layer_3_post_att_layer_norm_scale
params/encoder_layer_4_multi_head_att_value_fc.b_0
params/encoder_layer_11_multi_head_att_value_fc.b_0
params/encoder_layer_9_multi_head_att_query_fc.b_0
params/encoder_layer_4_multi_head_att_key_fc.b_0
params/encoder_layer_8_post_ffn_layer_norm_bias
params/encoder_layer_1_post_att_layer_norm_bias
params/encoder_layer_6_ffn_fc_0.w_0
params/encoder_layer_1_multi_head_att_output_fc.b_0
params/encoder_layer_5_multi_head_att_output_fc.b_0
params/encoder_layer_7_multi_head_att_output_fc.b_0
params/encoder_layer_8_post_ffn_layer_norm_scale
params/encoder_layer_0_multi_head_att_key_fc.w_0
params/encoder_layer_11_multi_head_att_key_fc.b_0
params/pooled_fc.b_0
params/encoder_layer_0_multi_head_att_query_fc.b_0
params/encoder_layer_9_ffn_fc_0.w_0
params/encoder_layer_8_ffn_fc_0.w_0
params/encoder_layer_9_post_att_layer_norm_scale
params/encoder_layer_3_multi_head_att_output_f

In [0]:
!git clone https://github.com/ArthurRizar/tensorflow_ernie

Cloning into 'tensorflow_ernie'...
remote: Enumerating objects: 34, done.
remote: Total 34 (delta 0), reused 0 (delta 0), pack-reused 34
Unpacking objects: 100% (34/34), done.


In [0]:
!python3 -m pip install paddlepaddle-gpu -i https://mirror.baidu.com/pypi/simple

Looking in indexes: https://mirror.baidu.com/pypi/simple
     |████████████████████████████████| 404.4MB 47kB/s 
     |████████████████████████████████| 122kB 4.0MB/s 
     |████████████████████████████████| 25.2MB 93kB/s 
  Created wheel for rarfile: filename=rarfile-3.1-cp36-none-any.whl size=24908 sha256=072ddd8fbc7aac632fb7bfb7750f3d0b00168971b0fa20480b46cd7a1c551645
  Stored in directory: /root/.cache/pip/wheels/b8/70/fe/66f580d6a6c698957c86d2316bf65c3cbb20e94cbbeccf1eb7
Successfully built rarfile
ERROR: kashgari 1.1.5 has requirement numpy==1.16.4, but you'll have numpy 1.18.4 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


## 实验结论：预训练模型在小样本情况下，有助于提升模型准确率

# 实验二：使用预训练bert语言模型，在不同粒度的预料库下进行模型fine-tuning，并在我们的标注问题上做评估

## 人民日报语料库

In [0]:
!pip install 'kashgari>=1.0.0,<2.0.0'
%tensorflow_version 1.x
from kashgari.corpus import ChineseDailyNerCorpus
from kashgari.tasks.labeling import BiLSTM_Model
import kashgari
kashgari.config.use_cudnn_cell = True

     |████████████████████████████████| 92kB 7.0MB/s 
     |████████████████████████████████| 17.3MB 189kB/s 
  Created wheel for keras-bert: filename=keras_bert-0.81.0-cp36-none-any.whl size=37913 sha256=6ff0467fd2af827ea8567bd336f227d647047583b5a17861f7f3f8ecbb584269
  Stored in directory: /root/.cache/pip/wheels/bd/27/da/ffc2d573aa48b87440ec4f98bc7c992e3a2d899edb2d22ef9e
  Created wheel for keras-gpt-2: filename=keras_gpt_2-0.14.0-cp36-none-any.whl size=10525 sha256=6b0388a5878b540bb644bab6ea4c9eeebe4366d49458eac194db831d1e154478
  Stored in directory: /root/.cache/pip/wheels/ec/d8/06/ba8216a77a55b8ba4a5c3932c7df93e87eeaea83ced27822aa
  Created wheel for bert4keras: filename=bert4keras-0.6.5-cp36-none-any.whl size=34212 sha256=b49d8dfe91f888a5042f5727c079c84e2d51aa70429e51cf808c634ecd9b392b
  Stored in directory: /root/.cache/pip/wheels/97/82/72/56a894ccb2337a25b679cbac552bf2f15f5e8e798a37313de6
  Created wheel for keras-transformer: filename=keras_transformer-0.33.0-cp36-none-any.w

TensorFlow 1.x selected.


In [0]:
kashgari.config.use_cudnn_cell = True

In [0]:
train_x, train_y = ChineseDailyNerCorpus.load_data('train')
test_x, test_y = ChineseDailyNerCorpus.load_data('test')
valid_x, valid_y = ChineseDailyNerCorpus.load_data('valid')

2449408/2443473 [==============================] - 2s 1us/step


In [0]:
def only_location(y):
  for yi in y:
    for i in range(len(yi)):
      label =  yi[i]
      if label == 'B-LOC' or label == 'I-LOC':
        pass
      else:
        yi[i] = 'O'
only_location(test_y)
only_location(valid_y)
only_location(train_y)

In [0]:
!wget https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip
!unzip chinese_L-12_H-768_A-12.zip -d chinese_L-12_H-768_A-12
import kashgari
from kashgari.tasks.labeling import BiLSTM_CRF_Model
from kashgari.embeddings import BERTEmbedding

bert_embed = BERTEmbedding('/content/chinese_L-12_H-768_A-12/chinese_L-12_H-768_A-12',
                           task=kashgari.LABELING,
                           sequence_length=100)
model = BiLSTM_CRF_Model(bert_embed)

--2020-05-07 13:30:12--  https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.126.128, 2a00:1450:4013:c01::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.126.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 381892918 (364M) [application/zip]
Saving to: ‘chinese_L-12_H-768_A-12.zip’

chinese_L-12_H-768_ 100%[===================>] 364.20M  93.1MB/s    in 3.9s    

2020-05-07 13:30:16 (93.1 MB/s) - ‘chinese_L-12_H-768_A-12.zip’ saved [381892918/381892918]

Archive:  chinese_L-12_H-768_A-12.zip
   creating: chinese_L-12_H-768_A-12/chinese_L-12_H-768_A-12/
  inflating: chinese_L-12_H-768_A-12/chinese_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: chinese_L-12_H-768_A-12/chinese_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: chinese_L-12_H-768_A-12/chinese_L-12_H-768_A-12/vocab.txt  
  inflating: chinese_L-12_H-768_A

/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:22: UserWarning: bert4keras.bert has been renamed as bert4keras.models.
  warnings.warn('bert4keras.bert has been renamed as bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:23: UserWarning: please use bert4keras.models.
  warnings.warn('please use bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:44: UserWarning: bert4keras.tokenizer has been renamed as bert4keras.tokenizers.
  warnings.warn('bert4keras.tokenizer has been renamed as bert4keras.tokenizers.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:45: UserWarning: please use bert4keras.tokenizers.
  warnings.warn('please use bert4keras.tokenizers.')


In [0]:
model.fit(train_x,
          train_y,
          x_validate=valid_x,
          y_validate=valid_y,
          epochs=50,
          batch_size=512)
model.save('/content/drive/My Drive/shiyan/02/bert_bilstm_crf-renmingribao2004.h5')

/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:22: UserWarning: bert4keras.bert has been renamed as bert4keras.models.
  warnings.warn('bert4keras.bert has been renamed as bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:23: UserWarning: please use bert4keras.models.
  warnings.warn('please use bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:44: UserWarning: bert4keras.tokenizer has been renamed as bert4keras.tokenizers.
  warnings.warn('bert4keras.tokenizer has been renamed as bert4keras.tokenizers.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:45: UserWarning: please use bert4keras.tokenizers.
  warnings.warn('please use bert4keras.tokenizers.')


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 100)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 100, 768), ( 16226304    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 100, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

KeyboardInterrupt: ignored

In [0]:
model.save('/content/drive/My Drive/shiyan/02/bert_bilstm_crf-renmingribao2004.h5')

In [0]:
model.evaluate(test_x,test_y)

           precision    recall  f1-score   support

      LOC     0.8825    0.9257    0.9036      3431

micro avg     0.8822    0.9257    0.9034      3431
macro avg     0.8825    0.9257    0.9036      3431



'           precision    recall  f1-score   support\n\n      LOC     0.8825    0.9257    0.9036      3431\n\nmicro avg     0.8822    0.9257    0.9034      3431\nmacro avg     0.8825    0.9257    0.9036      3431\n'

In [0]:
test_x_our

In [0]:
test_y_our_fixed = []

for yi in test_y_our:
  t = []
  for i in range(len(yi)):
    if yi[i]=='B-address':
      t.append('B-LOC')
    elif yi[i]=='I-address':
      t.append('I-LOC')
    else:
      t.append('O')
  test_y_our_fixed.append(t)
test_y_our_fixed

[['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'I-LOC',
  'I-LOC',
  'I-LOC',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'I-LOC',
  'I-LOC',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'I-LOC',
  'I-LOC',
  'I-LOC',
  'I-LOC',
  'I-LOC',
  'I-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O

In [0]:
model.evaluate(test_x_our,test_y_our_fixed)

           precision    recall  f1-score   support

      LOC     0.1095    0.2340    0.1492        94

micro avg     0.1095    0.2340    0.1492        94
macro avg     0.1095    0.2340    0.1492        94



'           precision    recall  f1-score   support\n\n      LOC     0.1095    0.2340    0.1492        94\n\nmicro avg     0.1095    0.2340    0.1492        94\nmacro avg     0.1095    0.2340    0.1492        94\n'

In [0]:
test_x_weijianwei,test_y_weijianwei = create_datas('/content/drive/My Drive/ourdata/weijianwei.json','test')
res = model.predict_entities(test_x_weijianwei)
get_entry(res)

['太 原 南',
 '临 汾',
 '临 汾 西',
 '太 原 市',
 '太 原 南',
 '阳 泉 市',
 '盂 县',
 '秀 水 镇',
 '路 洞',
 '区',
 '香 乐 村',
 '万 福 隆 超 市',
 '山 西 省',
 '平 遥 县',
 '香 乐 乡',
 '香 乐 村',
 '尖 草 坪 区',
 '汇 丰 街 道',
 '兴 华 北 小 区',
 '武 汉',
 '平 遥',
 '武 汉',
 '武 汉',
 '太 原',
 '太 原',
 '太 原',
 '临 汾 市',
 '吉 县',
 '苑',
 '尖 草 坪 区',
 '汇 丰 街 道',
 '兴 华 北 小 区',
 '沁 水 县',
 '郑 庄 镇',
 '中 乡 村',
 '临',
 '吉 高 速',
 '吉 县',
 '平 遥 县']

### 我们数据集

In [0]:
path = '/content/drive/My Drive/ourdata/ourtrain.train'
ws = []
ls = []
s= ''
with open(path,'r+',encoding="gbk") as f:
  s = f.read()
d = s.split('\n')
for di in d:
  sbi = di.split(' ')
  if len(sbi) == 2:
    w,l = sbi
    ws.append(w)
    ls.append(l)

In [0]:
ls

In [0]:
fenju = ['。','？','！']

In [0]:
w1 = []
l1 = []
wi = []
li = []

for w,l in zip(ws,ls):
  wi.append(w)
  li.append(l)
  if w in fenju or len(wi) >=200:
    w1.append(wi)
    l1.append(li)
    wi = []
    li = []

In [0]:
# dataname = ['train','dev','test']
# datapath = '/content/drive/My Drive/ourdata/our'
# words = []
# labels = []

# for datai in dataname:

path  = '/content/drive/My Drive/ourdata/allin_test.test'
ws = []
ls = []
s= ''
try:
  f = open(path,'r+',encoding="gbk")
  s = f.read()
except Exception as e:
  f = open(path,'r+',encoding="utf-8")
  s = f.read()
f.close()
d = s.split('\n')
for di in d:
  sbi = di.split(' ')
  if len(sbi) == 2:
    w,l = sbi
    ws.append(w)
    ls.append(l)
test_x = ws
test_y = ls

# w1 = []
# l1 = []
# wi = []
# li = []

# fenju = []

# for w,l in zip(ws,ls):
#   wi.append(w)
#   li.append(l)
#   if w in fenju or len(wi) >=200:
#     w1.append(wi)
#     l1.append(li)
#     wi = []
#     li = []
# words.append(w1)
# labels.append(l1)
  # eval(str(datai)+'_x') = w1
  # eval(str(datai)+'_y') = l1

In [0]:
train_x,dev_x,test_x = words
train_y,dev_y,test_y = labels

In [0]:
# x_data = makedata('疫情通报：。番禺区：小明在确诊病症后，居住于华南理工大学C10学生宿舍，活动于学生第一食堂。')
x_data = makedata('小明住在广东省深圳市宝安区西乡街道，确诊之后经常活动于好又多超市')

In [0]:
y = model.predict(x_data)

## MSRA

## boson语料库

In [0]:
!git clone https://github.com/7125messi/ner-pytorch

Cloning into 'ner-pytorch'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 69 (delta 29), reused 55 (delta 18), pack-reused 0
Unpacking objects: 100% (69/69), done.


In [0]:
import pickle
import pdb
with open('/content/ner-pytorch/data/Bosondata.pkl', 'rb') as inp:
    word2id = pickle.load(inp)
    id2word = pickle.load(inp)
    tag2id = pickle.load(inp)
    id2tag = pickle.load(inp)
    x_train = pickle.load(inp)
    y_train = pickle.load(inp)
    x_test = pickle.load(inp)
    y_test = pickle.load(inp)
    x_valid = pickle.load(inp)
    y_valid = pickle.load(inp)
print("train len:",len(x_train))
print("test len:",len(x_test))
print("valid len", len(x_valid))

train len: 10721
test len: 3351
valid len 2681


In [0]:
id2tag[1]

''

In [0]:
x_valid[88]

array([219, 378,  13,   7, 423,  37, 748,  93,  72,  30,  58,   1,  37,
       748,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0])

In [0]:
id2tag[0] = ''
id2tag[1] = 'E_location'

In [0]:
id2tag

0                   
1         E_location
2     M_product_name
3         E_org_name
4         B_org_name
5                  O
6     E_product_name
7         M_location
8     B_company_name
9      M_person_name
10        B_location
11     E_person_name
12     B_person_name
13    B_product_name
14        M_org_name
15    E_company_name
16            E_time
17    M_company_name
18            B_time
19            M_time
dtype: object

In [0]:
def transform_word(data):
  datanew = []
  for datai in  data:
    datanmewi = []
    for i in range(len(datai)):
      if datai[i] ==0:
        break
      datanmewi.append(id2word[datai[i]])
    datanew.append(datanmewi)
  return datanew

In [0]:
def fixed_label(data):
  labels = []
  for datai in data:
    label = []
    for i in range(len(datai)):
      if datai[i] == 0:
        break
      s = 'O'
      if 'location' in id2tag[datai[i]]:
        s =  id2tag[datai[i]]
        if s !=  'B_location':
          s = 'B-LOC'
        else:
          s = 'I-LOC'
      label.append(s)
    labels.append(label)
  return labels

In [0]:
id2tag

0                   
1         E_location
2     M_product_name
3         E_org_name
4         B_org_name
5                  O
6     E_product_name
7         M_location
8     B_company_name
9      M_person_name
10        B_location
11     E_person_name
12     B_person_name
13    B_product_name
14        M_org_name
15    E_company_name
16            E_time
17    M_company_name
18            B_time
19            M_time
dtype: object

In [0]:
x_trainm = transform_word(x_train)
x_testm = transform_word(x_test)
x_validm = transform_word(x_valid)

In [0]:
y_trainm = fixed_label(y_train)
y_testm = fixed_label(y_test)
y_validm = fixed_label(y_valid)

In [0]:
def fixedMSRA(x_data,y_data):
  res = []
  for x,y in zip(x_trainm,y_trainm):
    if 'B-LOC' not in y and 'I-LOC' not in y:
      pass
    else:
      res.append([x,y])
  xt = []
  yt = []
  for ri in res:
    xt.append(ri[0])
    yt.append(ri[1])
  # res
  return xt,yt

In [0]:
x_trainm1,y_trainm1 = fixedMSRA(x_trainm,y_trainm)

In [0]:
x_testm1,y_testm1 = fixedMSRA(x_testm,y_testm)

In [0]:
x_validm1,y_validm1 = fixedMSRA(x_validm,y_validm)

In [0]:
bert_embed = BERTEmbedding('/content/chinese_L-12_H-768_A-12/chinese_L-12_H-768_A-12',
                           task=kashgari.LABELING,
                           sequence_length=100)
model = BiLSTM_CRF_Model(bert_embed)

In [0]:
len(x_trainm1)

2670

In [0]:
len(x_validm1)

2670

In [0]:
model.fit(x_trainm1,y_trainm1,x_validm1,y_validm1,epochs=10)

model.fit(x_trainm1,
          y_trainm1,
          x_validate=x_validm1,
          y_validate=y_validm1,
          epochs=10,
          batch_size=512)
# model.save('/content/drive/My Drive/shiyan/02/bert_bilstm_crf-renmingribao2004.h5')

In [0]:
model.evaluate(x_testm1,y_testm1)

           precision    recall  f1-score   support

      LOC     0.1142    0.1164    0.1153      5180

micro avg     0.0763    0.1164    0.0922      5180
macro avg     0.1142    0.1164    0.1153      5180



'           precision    recall  f1-score   support\n\n      LOC     0.1142    0.1164    0.1153      5180\n\nmicro avg     0.0763    0.1164    0.0922      5180\nmacro avg     0.1142    0.1164    0.1153      5180\n'

In [0]:
test_x_weijianwei,test_y_weijianwei = create_datas('/content/drive/My Drive/ourdata/weijianwei.json','test')
model.predict_entities(test_x_weijianwei)
get_entry(res)

In [0]:
model.evaluate(test_x_our,test_y_our_fixed)

           precision    recall  f1-score   support

      LOC     0.0000    0.0000    0.0000        94

micro avg     0.0000    0.0000    0.0000        94
macro avg     0.0000    0.0000    0.0000        94



'           precision    recall  f1-score   support\n\n      LOC     0.0000    0.0000    0.0000        94\n\nmicro avg     0.0000    0.0000    0.0000        94\nmacro avg     0.0000    0.0000    0.0000        94\n'

# 